In [1]:
## Install Dependencies
!pip install numpy scikit-learn hyperopt plotly

In [2]:
## install optuna
!pip install git+https://github.com/optuna/optuna.git

  Cloning https://github.com/optuna/optuna.git to c:\users\vaiv\appdata\local\temp\pip-req-build-g7kg4_dg
  Resolved https://github.com/optuna/optuna.git to commit 2d9ee573fd232d7a8fd86e1a442dd57e13244cf0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/optuna/optuna.git 'C:\Users\vaiv\AppData\Local\Temp\pip-req-build-g7kg4_dg'


In [169]:
## Import Dependencies
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

## import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV,\
    RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
## import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
## import optuna
import optuna

## Dataset preparation

In [170]:
## load iris data
iris_data = load_iris()

## label, data for training 
label = iris_data.target
data = iris_data.data

X_train, X_val, y_train, y_val = train_test_split(data, label, test_size=0.2)

## 1. Manual Search
### : Method to find out the optimal hyperparameter manually

#### criterion = the function used to evaluate the quality of a split. 
#### max_depth = maximum number of levels allowed in each tree.
#### max_features = maximum number of features considered when splitting a node.
#### min_samples_leaf = minimum number of samples which can be stored in a tree leaf.
#### min_samples_split = minimum number of samples necessary in a node to cause node splitting.
#### n_estimators = number of trees in the ensemble.

In [176]:
## manual model 1
manual1 = RandomForestClassifier(random_state= 101).fit(X_train,y_train)
pred = manual1.predict(X_val)
man1_acc = accuracy_score(y_val, pred)
print("manual model 1 performance : ", man1_acc)

manual model 1 performance :  0.9666666666666667


In [177]:
## manual model 2 (add n_estimators)
manual2 = RandomForestClassifier(max_depth= 20, n_estimators= 20, random_state= 101, min_samples_split=10).fit(X_train,y_train)
pred = manual2.predict(X_val)
man2_acc = accuracy_score(y_val, pred)
print("manual model 2 performance : ", man2_acc)

manual model 2 performance :  1.0


## Setting parameter for GridSearchCV

In [12]:
# setting param_grid for GridSearchCV
param_grid = {'max_depth': np.arange(5,8,1),
             'n_estimators':np.arange(130,170,10)}

In [13]:
## load model instance
## for practicing, use Gradient Boosting classifier model
gbc = GradientBoostingClassifier()

## 2. GridSearchCV

In [14]:
param_grid = {
    'n_estimators': np.arange(10, 201, 20), 
    'max_depth':np.arange(3, 15, 2),
    'criterion': ['entropy', 'gini'],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': np.arange(4, 8, 2)
    }
rfc = RandomForestClassifier()
rfc_grid = GridSearchCV(rfc, param_grid=param_grid, n_jobs=8)
rfc_grid.fit(X_train, y_train)

## check out best parameters and scores
print('best parameters of random forest classifier : ', rfc_grid.best_params_)
print('best score of random forest classifier : ', rfc_grid.best_score_)

best parameters of random forest classifier :  {'criterion': 'entropy', 'max_depth': 9, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'n_estimators': 10}
best score of random forest classifier :  0.9916666666666668


In [15]:
## setting best_estimator_model
em = rfc_grid.best_estimator_
## predict
pred = em.predict(X_val)

print("accuracy performance of best estimator : ", accuracy_score(y_val, pred))

accuracy performance of best estimator :  0.8333333333333334


## 3. RandomSearchCV

In [16]:
param_grid= {
    'n_estimators': np.arange(10, 201, 20), 
    'max_depth':np.arange(3, 15, 2),
    'criterion': ['entropy', 'gini'],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': np.arange(4, 8, 2)
    }

rfc= RandomForestClassifier()
random= RandomizedSearchCV(rfc, param_distributions= param_grid,
                            n_iter= 10, cv= 3, verbose= 1, 
                            random_state= 101, n_jobs= -1)
random.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': array([ 3,  5,  7,  9, 11, 13]),
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': array([4, 6]),
                                        'n_estimators': array([ 10,  30,  50,  70,  90, 110, 130, 150, 170, 190])},
                   random_state=101, verbose=1)

In [17]:
## predict
random_pf= random.best_estimator_.predict(X_val)
random_acc= accuracy_score(y_val, random_pf)

print(random.best_params_)
print(random_acc)

{'n_estimators': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 9, 'criterion': 'gini'}
0.9


## 4. HyperOpt

#### parameter of fmin method
#### objective function: loss function to minimize
#### Domain space: probability distribution of hyperparameter
#### optimization algorithm: optimal combination algorithm

In [18]:
# space: 알고리즘이 탐색할 범위를 정의한다. 
# hp.choice: 리스트 내의 값을 무작위로 추출
# hp.uniform: 정의된 범위 내에서 임의의 숫자를 무작위 추출
# hp.quniform: 정의된 범위 내에서 마지막 숫자만큼의 간격을 두어 숫자를 추출 
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.choice('max_depth', np.arange(3, 15, 2, dtype=int)),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50])
    }

def objective(space):
    hopt = RandomForestClassifier(criterion = space['criterion'], 
                                   max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(hopt, X_train, y_train, cv = 4).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20,
            trials= trials)

 40%|██████████               | 8/20 [00:01<00:02,  4.71trial/s, best loss: -0.9166666666666666]

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly s

 50%|████████████            | 10/20 [00:02<00:02,  4.97trial/s, best loss: -0.9166666666666666]

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(



100%|████████████████████████| 20/20 [00:04<00:00,  4.43trial/s, best loss: -0.9416666666666667]


In [19]:
# 최적의 하이퍼파라미터 조합
best

{'criterion': 0,
 'max_depth': 3,
 'max_features': 2,
 'min_samples_leaf': 0.011900434509813707,
 'min_samples_split': 0.4151822864537602,
 'n_estimators': 1}

In [20]:
# best를 통해 최적의 조합이 도출되었으니, 이를 바탕으로 모델을 학습시켜야 한다. 그러기 위해서는 변수들이 가진 element에 인덱스를 부여한다. 
# hp.uniform은 숫자를 무작위로 추출하기에 우리가 알 수 없으므로, choice, quniform을 메소드로 가진 변수들만 딕셔너리로 만들어준다. 
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 75, 3: 100, 4: 125}

trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], 
                                      max_depth = best['max_depth'], 
                                      max_features = feat[best['max_features']], 
                                      min_samples_leaf = best['min_samples_leaf'], 
                                      min_samples_split = best['min_samples_split'], 
                                      n_estimators = est[best['n_estimators']]
                                     ).fit(X_train,y_train)

In [21]:
hopt_pf= trainedforest.predict(X_val)
hopt_acc = accuracy_score(y_val, hopt_pf)
print(hopt_acc)

0.8333333333333334


## 5. Optuna

In [140]:
import optuna
import sklearn
from optuna.samplers import TPESampler

In [147]:
# 1. 최소화/최대화할 목적함수 정의
def objective(trial: Trial, data, label):
    # 2. trial object로 하이퍼파라미터 값 추천
    rf_param= {
                'n_estimators': trial.suggest_int('n_estimators', 10, 200), 
                'max_depth':trial.suggest_int('max_depth', 3, 15),
                'criterion': trial.suggest_categorical('criterion', ['entropy', 'gini']),
                'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt']),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 4, 8)
                }
    classifier_obj = sklearn.ensemble.RandomForestClassifier(**rf_param)
    
    accuracy = cross_val_score(classifier_obj, data, label, cv = 4).mean()
    return accuracy

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study(direction='maximize', sampler=TPESampler())
# 반복 시행 횟수(trial)는 200번으로
study.optimize(lambda trial: objective(trial, data, label), n_trials=200)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2022-10-21 15:22:57,572] A new study created in memory with name: no-name-72db9a10-d8f8-4648-b55f-8e47883b5b64
C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='a

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

[I 2022-10-21 15:23:01,751] Trial 6 finished with value: 0.9466571834992887 and parameters: {'n_estimators': 107, 'max_depth': 7, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 7}. Best is trial 1 with value: 0.9667496443812233.
[I 

[I 2022-10-21 15:23:06,948] Trial 17 finished with value: 0.9601706970128022 and parameters: {'n_estimators': 37, 'max_depth': 15, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 6}. Best is trial 1 with value: 0.9667496443812233.
C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:

[I 2022-10-21 15:23:10,074] Trial 23 finished with value: 0.9532361308677098 and parameters: {'n_estimators': 87, 'max_depth': 5, 'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9667496443812233.
C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.



C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_featu

...

[I 2022-10-21 15:24:11,096] Trial 180 finished with value: 0.9599928876244666 and parameters: {'n_estimators': 21, 'max_depth': 5, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 4}. Best is trial 162 with value: 0.9733285917496444.
C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

[I 2022-10-21 15:24:13,091] Trial 186 finished with value: 0.9667496443812233 and parameters: {'n_estimators': 65, 'max_depth': 5, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 4}. Best is trial 162 with value: 0.973328591749644

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_featu

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_featu

Best trial : score 0.9733285917496444, 
params {'n_estimators': 65, 'max_depth': 6, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 4}


In [148]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)

{'n_estimators': 65, 'max_depth': 6, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 4}
0.9733285917496444


In [149]:
# 하이퍼파라미터별 중요도
optuna.visualization.plot_param_importances(study)

In [150]:
# 하이퍼파라미터 최적화 과정
optuna.visualization.plot_optimization_history(study)

## Compare sorts of classifiers

In [151]:
# 1. 최소화/최대화할 목적함수 정의
def objective(trial: Trial, data, label):

    # 2. trial object로 하이퍼파라미터 값 추천
    
    classifier_name= trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    #분류 모델이 SVC일 때
    if classifier_name == 'SVC':
        svc_param= {
                    'C': trial.suggest_loguniform('C', 1e-10, 1e10),
                    'gamma': trial.suggest_categorical('gamma', ['auto', 'scale']),
                    'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf'])
                    }
        
        classifier_obj = sklearn.svm.SVC(**svc_param)
    
    #분류모델이 랜덤포레스트일 때
    else:
        rf_param= {
                    'n_estimators': trial.suggest_int('n_estimators', 10, 200), 
                    'max_depth':trial.suggest_int('max_depth', 3, 15),
                    'criterion': trial.suggest_categorical('criterion', ['entropy', 'gini']),
                    'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt']),
                    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 4, 8)
                    }
        classifier_obj = sklearn.ensemble.RandomForestClassifier(**rf_param)
    
    accuracy = cross_val_score(classifier_obj, data, label, cv = 4).mean()
    return accuracy

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study(direction='maximize', sampler=TPESampler())
# 반복 시행 횟수(trial)는 200번으로
study.optimize(lambda trial: objective(trial, data, label), n_trials=200)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2022-10-21 15:26:42,906] A new study created in memory with name: no-name-53e5668f-1eef-463c-95ce-98d1fac81613
C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='a

C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

[I 2022-10-21 15:26:46,123] Trial 6 finished with value: 0.9601706970128022 and parameters: {'classifier': 'RandomForest', 'n_estimators': 160, 'max_depth': 7, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 7}. Best is trial 6 with value: 0.9601706970128022.
C:\Users\vaiv\Documents\Git\ConvAE_LSTM\.convAE_lstm\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers a

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:26:50,802] Trial 17 finished with value: 0.9665718349928876 and parameters: {'classifier': 'SVC', 'C': 0.9143910082970645, 'gamma': 'auto', 'kernel': 'rbf'}. Best is trial 16 with value: 0.9733285917496444.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:26:50,819] Trial 18 finished with value: 0.4822190611664296 and parameters: {'classifier': 'SVC', 'C': 3.946406831039078e-05, 'gamma': 'scale', 'kernel': 'linear'}. Best is trial 16 with 

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:26:51,880] Trial 33 finished with value: 0.9535917496443812 and parameters: {'classifier': 'SVC', 'C': 1018540.4038686302, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 22 with value: 0.9800853485064012.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:26:51,899] Trial 34 finished with value: 0.9799075391180655 and parameters: {'classifier': 'SVC', 'C': 162.2580276364283, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 22 with valu

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:26:54,105] Trial 51 finished with value: 0.9799075391180655 and parameters: {'classifier': 'SVC', 'C': 143.75089150159286, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 22 with value: 0.9800853485064012.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:26:54,122] Trial 52 finished with value: 0.9733285917496444 and parameters: {'classifier': 'SVC', 'C': 5.272278433657963, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 22 with valu

...

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:00,932] Trial 146 finished with value: 0.9733285917496444 and parameters: {'classifier': 'SVC', 'C': 12.509644676421324, 'gamma': 'auto', 'kernel': 'poly'}. Best is trial 22 with value: 0.9800853485064012.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:00,954] Trial 147 finished with value: 0.9799075391180655 and parameters: {'classifier': 'SVC', 'C': 104.6801174821736, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 22 with val

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:01,285] Trial 162 finished with value: 0.9731507823613087 and parameters: {'classifier': 'SVC', 'C': 532.011704431613, 'gamma': 'scale', 'kernel': 'linear'}. Best is trial 22 with value: 0.9800853485064012.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:01,309] Trial 163 finished with value: 0.9731507823613087 and parameters: {'classifier': 'SVC', 'C': 12.881279074671438, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 22 with v

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:02,441] Trial 176 finished with value: 0.9735064011379801 and parameters: {'classifier': 'SVC', 'C': 0.02140635280401711, 'gamma': 'auto', 'kernel': 'poly'}. Best is trial 22 with value: 0.9800853485064012.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:02,466] Trial 177 finished with value: 0.9799075391180655 and parameters: {'classifier': 'SVC', 'C': 5.643309346298721, 'gamma': 'auto', 'kernel': 'poly'}. Best is trial 22 with val

C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:03,156] Trial 192 finished with value: 0.966394025604552 and parameters: {'classifier': 'SVC', 'C': 104.44457641320949, 'gamma': 'scale', 'kernel': 'linear'}. Best is trial 22 with value: 0.9800853485064012.
C:\Users\vaiv\AppData\Local\Temp\ipykernel_78776\803344188.py:10: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-10-21 15:27:03,177] Trial 193 finished with value: 0.9731507823613087 and parameters: {'classifier': 'SVC', 'C': 823.3485905859053, 'gamma': 'auto', 'kernel': 'linear'}. Best is trial 22 with 

Best trial : score 0.9800853485064012, 
params {'classifier': 'SVC', 'C': 6.90215330917403, 'gamma': 'auto', 'kernel': 'linear'}


In [155]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)

{'classifier': 'SVC', 'C': 6.90215330917403, 'gamma': 'auto', 'kernel': 'linear'}
0.9800853485064012


In [156]:
# 하이퍼파라미터별 중요도
optuna.visualization.plot_param_importances(study)

In [157]:
# 하이퍼파라미터 최적화 과정
optuna.visualization.plot_optimization_history(study)